# The Aer device  
The ```qiskit.aer``` divice provided by the PennyLane-Qiskit plugin allows you to see PennyLane to deploy and run your quantum machine learning models on the backends and simulators provided by Qiskit Aer

In [1]:
import pennylane as qml
dev = qml.device('qiskit.aer', wires = 2)

This device can then be used just like other devices for the definition and evaluation of QNodes within PennyLane. A simple quantum function that returns the expectation value of a measurement and depends on three classical input parameters would look like:

In [8]:
@qml.qnode(dev)
def circuit(x,y,z):
    qml.RZ(z, wires=0)
    qml.RY(y, wires=0)
    qml.RX(x, wires=0)
    qml.CNOT(wires=[0,1])

    return qml.expval(qml.Z(1))

circuit(0.2, 0.1, 0.3)

array(0.96289062)

## Backend Methods and Options  
The default backend is the ```AerSimulator```. However, multiple other backends are also available. To get a current overview what backends are available you can query  

Note   
Currently, PennyLane does not support the ```pulse_simulator```

In [9]:
from qiskit_aer import Aer
Aer.backends()


[AerSimulator('aer_simulator'),
 AerSimulator('aer_simulator_statevector'),
 AerSimulator('aer_simulator_density_matrix'),
 AerSimulator('aer_simulator_stabilizer'),
 AerSimulator('aer_simulator_matrix_product_state'),
 AerSimulator('aer_simulator_extended_stabilizer'),
 AerSimulator('aer_simulator_unitary'),
 AerSimulator('aer_simulator_superop'),
 QasmSimulator('qasm_simulator'),
 StatevectorSimulator('statevector_simulator'),
 UnitarySimulator('unitary_simulator')]

You can change a ```qiskit.aer``` device's backend with the ```backend``` argument when creating the ```device```

In [11]:
from qiskit_aer import UnitarySimulator, AerSimulator
dev = qml.device('qiskit.aer', wires = 2, backend = UnitarySimulator())
dev = qml.device('qiskit.aer', wires = 2, backend = AerSimulator())

Note  
Occasionally, you may see others pass in a string as a backend. Look below for example. However, this will soon be deprecated and may not funciton as intended

In [14]:
dev = qml.device('qiskit.aer', wires = 2, backend = 'aer_simulator_matrix_product_state')

The ```AerSimulator``` backend has several available methods, which can be passed via the ```method``` keyword argument. For example ```automatic```, ```statevector```, and ```unitary```

In [15]:
dev = qml.device('qiskit.aer', wires = 2, backend = AerSimulator(), method ='automatic')

Each of these methods can take different *run* options, for example to specify the numerical precision of the simulation.

In [16]:
dev = qml.device('qiskit.aer', wires=2, backend=AerSimulator(), validation_threshold=1e-6)

## Noise Models  
One great feature of the ```qiskit.aer``` device is the ability to simulate noise. There are different noise models, which you can instantiate and apply to the devices as follows

In [31]:
import pennylane as qml
import qiskit
from qiskit_aer import noise

prob_1 = 0.001
prob_2 = 0.01

error_1 = noise.depolarizing_error(prob_1, 1) # 숫자는 이 오류가 적용될 큐비트의 개수
error_2 = noise.depolarizing_error(prob_2, 2)

noise_model = noise.NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1, ['u1','u2','u3'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

dev = qml.device('qiskit.aer', wires = 2, noise_model = noise_model)

@qml.qnode(dev)
def circuit(x, y, z):
    qml.RZ(z, wires=[0])
    qml.RY(y, wires=[0])
    qml.RX(x, wires=[0])
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(wires=1))

# Result of noisy simulator
print(circuit(0.2, 0.1, 0.3))

0.962890625
